In [5]:
import pandas as pd
import json
data = pd.read_csv("/Users/isaac/Desktop/SI 485/clearinghouse_case_links_by_collections(without 5679 and 5666).csv")
data.head()
data['Link'].to_list()
with open("cases_by_court.json", "w") as f:
    json.dump(data['Link'].to_list(), f)

In [2]:
#function that extracts a list from the json file of all the case numbers:
import json
def extract_case_numbers(json_file):
    with open(json_file, 'r') as f:
        links = json.load(f)
    
    case_numbers = []
    for link in links:
        # Assuming the case number is the last part of the URL after 'case='
        if 'case=' in link:
            case_number = link.split('case=')[-1]
            case_numbers.append(case_number)
    
    return case_numbers
#extract_case_numbers('cases_by_collections.json')
numbers_as_int = [int(num) for num in extract_case_numbers('cases_by_collections.json')]
print(numbers_as_int)

[17571, 17539, 17201, 16827, 16822, 16200, 16016, 15931, 15930, 15202, 14457, 14417, 14054, 11854, 11750, 11703, 11435, 11418, 11365, 11358, 11310, 11302, 11295, 11294, 11253, 11228, 11029, 11028, 10868, 10689, 10616, 10615, 10614, 10586, 10558, 10555, 10554, 10553, 10530, 10518, 10517, 10378, 10268, 10264, 10260, 10259, 10258, 10257, 10256, 10255, 10254, 10251, 10249, 10246, 10165, 10083, 10035, 10033, 10032, 10031, 10026, 10024, 10023, 10022, 10021, 10019, 10018, 10017, 10015, 10014, 10011, 10010, 10009, 10008, 10007, 10006, 10005, 10004, 10002, 10001, 10000, 9999, 9998, 9996, 9995, 9992, 9987, 9986, 9984, 9983, 9980, 9979, 9978, 9975, 9974, 9973, 9972, 9971, 9970, 9969, 9968, 9963, 9961, 9960, 9959, 9958, 9957, 9954, 9951, 9950, 9949, 9948, 9945, 9944, 9943, 9942, 9899, 9894, 9893, 9890, 9872, 9871, 9870, 9869, 9868, 9867, 9866, 9865, 9863, 9860, 9859, 9857, 9854, 9852, 9850, 9849, 9847, 9845, 9844, 9842, 9840, 9839, 9837, 9836, 9835, 9834, 9833, 9832, 9831, 9830, 9829, 9828, 9827, 

In [11]:
import csv
import json
import re
import time
import requests

API_TOKEN = "11a7d2a0a5c673e0d4391cb578563eb43d629a49"
HEADERS = {"Authorization": f"Token {API_TOKEN}", "User-Agent": "Chrome v22.2 Linux Ubuntu"}
BASE_URL = "https://clearinghouse.net/api/v2/cases/"
CSV_PATH = "drive-download-20250909T014355Z-1-001/clearinghouse_case_links_by_collections(without 5679 and 5666).csv"
OUTPUT_JSON = "filtered_cases.json"

def case_ids_from_csv(path):
    with open(path, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            match = re.search(r"case=(\d+)", row["Link"])
            if match:
                yield int(match.group(1))

def fetch_case(case_id, max_retries=5, base_delay=1.5):
    delay = base_delay
    for attempt in range(1, max_retries + 1):
        resp = requests.get(
            BASE_URL,
            params={"case_id": case_id},
            headers=HEADERS,
            timeout=30,
        )

        if resp.status_code == 200:
            items = resp.json().get("results", [])
            return items[0] if items else None

        if resp.status_code == 429:
            print(f"{case_id}: hit rate limit, sleeping {delay:.1f}s (attempt {attempt}/{max_retries})")
        elif 500 <= resp.status_code < 600:
            print(f"{case_id}: server error {resp.status_code}, retrying in {delay:.1f}s (attempt {attempt}/{max_retries})")
        else:
            resp.raise_for_status()

        time.sleep(delay)
        delay *= 2

    raise RuntimeError(f"Unable to fetch case {case_id} after {max_retries} retries")


summaries = []

for cid in case_ids_from_csv(CSV_PATH):
    data = fetch_case(cid)
    if not data:
        print(f"No data for {cid}")
        continue

    summary = {
        "case_id": cid,
        "case_name": data.get("name"),
        "court": data.get("court"),
        "defendants": [d.get("name") for d in data.get("case_defendants", [])],
        "defendant_types": [
            d.get("defendant_type") or d.get("defendant_types") for d in data.get("case_defendants", [])
        ],
        "facility_types": data.get("facility_type", []),
        "causes_of_action": data.get("causes", []),
        "constitutional_clauses": [c.get("value") for c in data.get("constitutional_clause", [])],
        "available_documents": data.get("available_documents", []),
    }
    summaries.append(summary)

    print(f"Collected case {cid}")
    time.sleep(0.2)  # polite pause

with open(OUTPUT_JSON, "w", encoding="utf-8") as fp:
    json.dump(summaries, fp, indent=2, ensure_ascii=False)

print(f"Saved {len(summaries)} cases to {OUTPUT_JSON}")


Collected case 17571
Collected case 17539
Collected case 17539
Collected case 17201
Collected case 17201
Collected case 16827
Collected case 16827
Collected case 16822
Collected case 16822
Collected case 16200
Collected case 16200
Collected case 16016
Collected case 16016
Collected case 15931
Collected case 15931
Collected case 15930
Collected case 15930
Collected case 15202
Collected case 15202
Collected case 14457
Collected case 14457
Collected case 14417
Collected case 14417
Collected case 14054
Collected case 14054
Collected case 11854
Collected case 11854
Collected case 11750
Collected case 11750
Collected case 11703
Collected case 11703
Collected case 11435
Collected case 11435
Collected case 11418
Collected case 11418
Collected case 11365
Collected case 11365
Collected case 11358
Collected case 11358
Collected case 11310
Collected case 11310
Collected case 11302
Collected case 11302
Collected case 11295
Collected case 11295
Collected case 11294
Collected case 11294
Collected cas